In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os #deal with directories and can work on all OS

In [ ]:
## Path definitions
PROJECT_ROOT = os.path.join(os.getcwd(), '..') #Main file (in this case "AI_Analysis_and_Chatbot")
EXCEL_FILE_PATH = os.path.join(PROJECT_ROOT, 'Data', 'amazon_product_reviews.xlsx') #gets path with file name and folder

In [ ]:
## Data cleaning
def clean_num (series): #remove symbols from numbers and and convert to float
    if series.dtype == 'object': #when reading it appears as text (so we dont take numbers without symbols)
        return (series.astype(str) #cast to string for consistency
                      .str.replace('â‚¹', '', regex=False) #delete symbol (replace with empty string)
                      .str.replace(',', '', regex=False) #so they can be dealt with as floats/ints not strings
                      .str.replace('%', '', regex=False)
                      .str.strip() #remove spaces before/after string
                      .astype(float, errors='coerce')) # Convert to float, turns errors (like empty strings) to NaN
    return series

SyntaxError: incomplete input (2877722192.py, line 5)